# Surfalex formability predictions - MatFlow workflow analysis

This Jupyter notebook demonstrates the MatFlow workflows that were generated to investigate the formability of the Surfalex HF Al alloy. Running the cells in this notebook demonstrates how the MatFlow Python API can be used to inspect the results from, and perform further analysis on, MatFlow workflows.

This notebook and the five associated MatFlow workflows can be considered supplementary data to the following manuscript:

***'A novel integrated framework for reproducible formability predictions using virtual materials testing'***, A. J. Plowman, P. Jedrasiak, T. Jailin, P. Crowther, S. Mishra, P. Shanthraj, J. Quinta da Fonseca, in preparation.

In the work described in the above manuscript, we split the workflow into five sub-workflows, as follows:

1. Generate a representative volume element (with DAMASK and MTEX)
2. Fit single-crystal parameters for calibrated crystal plasticity (CP) simulations (with DAMASK)
3. Fit yield functions from full field CP simulations (with DAMASK and formable)
4. Estimate hardening curves from full field CP simulations (with DAMASK)
5. Perform Marciniak-Kukcyski simulations using the finite-element method (with Abaqus)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import utilities

## Import packages

In [ ]:
import pathlib
import dic_analysis.io
import tqdm
import urllib.request
import hashlib

from matflow import load_workflow
from formable.load_response import LoadResponse, LoadResponseSet
from formable.levenberg_marquardt import LMFitter
from formable.tensile_test import TensileTest
from formable.yielding import YieldFunction, animate_yield_function_evolution
from plotly import graph_objects
from plotly.colors import DEFAULT_PLOTLY_COLORS
import pandas as pd
import numpy as np
from IPython.display import display
from ipywidgets import widgets

## Utility functions

In [ ]:
FIG_EXPORT_DIR = pathlib.Path('generated_figures')
FIG_EXPORT_DIR.mkdir(exist_ok=True)

PLASTIC_TABLES_DIR = Path('generated_plastic_tables')
PLASTIC_TABLES_DIR.mkdir(exist_ok=True)

SHEET_DIRS = {'x': 'RD', 'y': 'TD', 'z': 'ND'}

In [ ]:
def pretty_print_single_crystal_parameters(parameters):
    'Pretty print the single crystal parameter dict'
    out = (
        f'Hardening coefficient,   h_0_sl_sl: {parameters["Al"]["h_0_sl_sl"]/1e6:>7.1f} MPa\n'
        f'Initial CRSS,              xi_0_sl: {parameters["Al"]["xi_0_sl"][0]/1e6:>7.1f} MPa\n'
        f'Final CRSS,              xi_inf_sl: {parameters["Al"]["xi_inf_sl"][0]/1e6:>7.1f} MPa\n'
        f'Hardening exponent,           a_sl: {parameters["Al"]["a_sl"]:>7.1f}\n'
    )
    print(out)
    
def plot_static_figure_single_crystal_fitting(lm_fitter):
    data = [
        {
            'x': lm_fitter.optimisations[0].get_exp_strain(),
            'y': lm_fitter.optimisations[0].get_exp_stress() / 1e6,
            'mode': 'lines',
            'name': 'Experimental',
            'line': {
                'dash': 'dash',
                'width': 3,
                'color': DEFAULT_PLOTLY_COLORS[0],
            },            
        },
        {
            'x': lm_fitter.optimisations[0].get_sim_strain(0),
            'y': lm_fitter.optimisations[0].get_sim_stress(0) / 1e6,
            'mode': 'lines',
            'line': {
                'dash': 'dot',
                'width': 1,
                'color': DEFAULT_PLOTLY_COLORS[1],
            },
            'name': 'Sim. initial iter.', 
        },
        {
            'x': lm_fitter.optimisations[-1].get_sim_strain(0),
            'y': lm_fitter.optimisations[-1].get_sim_stress(0) / 1e6,
            'mode': 'lines',
            'name': 'Sim. final iter.',
            'line': {
                'width': 1,
                'color': DEFAULT_PLOTLY_COLORS[1],
            },            
        }
    ]
    layout = {
        'width': 280,
        'height': 250,
        'margin': {'t': 20, 'b': 20, 'l': 20, 'r': 20},
        'template': 'simple_white',
        'xaxis': {
            'title': r'True strain, \strain{}',
            'range': [
                -0.01,
                lm_fitter.optimisations[0].get_exp_strain()[-1],
            ],
            'mirror': 'ticks',
            'ticks': 'inside',
            'dtick': 0.05,
            'tickformat': '.2f',
        },
        'yaxis': {
            'title': r'True stress, \stress{} (\MPa)',
            'mirror': 'ticks',
            'ticks': 'inside',
            'range': [0, 310],
        },
        'legend': {
            'x': 0.95,
            'y': 0.05,
            'xanchor': 'right',
            'yanchor': 'bottom',
            'tracegroupgap': 0,
        },
    }
    fig = graph_objects.Figure(data=data, layout=layout)    
    return fig

def plot_static_figure_yield_function_type_comparison(all_load_responses, yield_point):    
    yld_funcs_subset = []
    for resp in all_load_responses:
        yield_function_idx = np.where(resp.yield_point_criteria[0].values[0] == yield_point)[0][0]
        yld_funcs_subset.append(
            resp.fitted_yield_functions[yield_function_idx]['yield_function']
        )    
    plane = [0, 0, 1]
    fig_wig = YieldFunction.compare_2D(yld_funcs_subset, plane)
    fig_wig.layout.annotations = () # remove annotations    
    
    line_styles = {
        'Hill1948': {'dash': 'dash'},
        'Barlat_Yld91': {'dash': 'dot'},
        'Barlat_Yld2004_18p': {'dash': 'solid'},
    }
    for idx, trace in enumerate(fig_wig.data):
        if 'VonMises' in trace.name:
            new_name = 'Von Mises'
        elif 'Barlat_Yld91' in trace.name:
            new_name = 'Barlat Yld91'
            trace.line.update(line_styles['Barlat_Yld91'])
        elif 'Barlat_Yld2000_2D' in trace.name:
            new_name = 'Barlat Yld2000-2D'
        elif 'Barlat_Yld2004_18p' in trace.name:
            new_name = 'Barlat Yld2004-18p'
            trace.line.update(line_styles['Barlat_Yld2004_18p'])
        elif 'Hill1979' in trace.name:
            new_name = 'Hill 1979'
        elif 'Hill1948' in trace.name:
            new_name = 'Hill 1948'
            trace.line.update(line_styles['Hill1948'])
        else:
            new_name = trace.name
        
        trace.name = new_name
        trace.line.update({
            'color': DEFAULT_PLOTLY_COLORS[idx],
            'width': 1,
        })
        
    fig_wig.layout.update({
        'width': 280,
        'height': 280,
        'margin': {'t': 20, 'b': 20, 'l': 20, 'r': 20},
        'template': 'simple_white',
        'xaxis': {
            'mirror': 'ticks',
            'ticks': 'inside',
            'range': [0.1, 1.2],
            'dtick': 0.2,
            'title': r'\yldFuncFigXLab{}',
            'tickformat': '.1f',
        },
        'yaxis': {
            'mirror': 'ticks',
            'ticks': 'inside',
            'range': [0.1, 1.2],
            'dtick': 0.2,
            'title': r'\yldFuncFigYLab{}',
            'tickformat': '.1f',
        },
        'legend': {
            'x': 0.07,
            'y': 0.1,
            'xanchor': 'left',
            'yanchor': 'bottom',
            'bgcolor': 'rgba(255, 255, 255, 0)',
            'tracegroupgap': 0,
        }
    })    
    return fig_wig

def compare_yield_function_yield_points(load_response_set, ypc_slice, **kwargs):
    yld_funcs_subset = []
    yld_points = []
    for i in load_response_set.fitted_yield_functions[ypc_slice]:
        yld_funcs_subset.append(i['yield_function'])
        yld_points.append(load_response_set.yield_point_criteria[i['YPC_idx']].values[0, i['YPC_value_idx']])         

    return YieldFunction.compare_2D(yld_funcs_subset, legend_text=yld_points, **kwargs)

def compare_yield_function_types(all_load_responses, yield_function_idx=0, **kwargs):
    yld_funcs_subset = [
        resp.fitted_yield_functions[yield_function_idx]['yield_function']
        for resp in all_load_responses
    ]
    
    return YieldFunction.compare_2D(yld_funcs_subset, **kwargs)    

def show_all_fitted_yield_function_parameters(all_load_responses):
    all_fitted_params = {}
    for yld_func_type_idx in all_load_responses:
        yld_func_type_fitted_params = []
        for i in yld_func_type_idx.fitted_yield_functions:
            yld_point = all_load_responses[1].yield_point_criteria[i['YPC_idx']].values[0, i['YPC_value_idx']]
            yld_func_type_fitted_params.append({'yield_point': yld_point, **i['yield_function'].get_parameters()})
        all_fitted_params.update({i['yield_function'].name: yld_func_type_fitted_params})

    tab_children = []
    tab_titles = []
    for yld_func_name, fitted_vals in all_fitted_params.items():
        df = pd.DataFrame(fitted_vals)    
        out_widget = widgets.Output()
        tab_children.append(out_widget)
        tab_titles.append(yld_func_name)
        with out_widget:
            display(df)

    tab = widgets.Tab()
    tab.children = tab_children
    for i in range(len(tab_titles)):
        tab.set_title(i, tab_titles[i])
    display(tab)
    return all_fitted_params

def collect_hardening_data(sim_tasks, yield_stress):
        
    # Collect interpolated stresses to feed to the FE Marciniak-Kuczynski model:
    strain_paths = list(sim_tasks.keys())
    interp_strains = np.linspace(0, 0.29, num=40)
    
    hardening_data = {k: None for k in strain_paths}
    for strain_path, sim_task in sim_tasks.items():    
        strain = sim_task.elements[0].outputs.volume_element_response['vol_avg_equivalent_plastic_strain']['data']
        stress = sim_task.elements[0].outputs.volume_element_response['vol_avg_equivalent_stress']['data']
        hardening_data[strain_path] = {
            'strain': strain,
            'stress': stress,
            'interpolated_strain': interp_strains,
            'interpolated_stress': np.interp(interp_strains, strain, stress),
            'hardening_rate': np.gradient(stress),
        }
        # Set the first interpolation value (zero-strain) to the approximate yield stress:
        hardening_data[strain_path]['interpolated_stress'][0] = yield_stress
        
    return hardening_data

def show_interpolated_plastic_stress_strain_data(hardening_data):
    interpolated_hardening_data = []
    column_headers = []
    for k, v in hardening_data.items():
        interpolated_hardening_data.append(v['interpolated_strain'])
        interpolated_hardening_data.append(v['interpolated_stress'])
        column_headers.extend([(k, 'interpolated_strain'), (k, 'interpolated_stress')])
    interpolated_hardening_data = np.array(interpolated_hardening_data)
    df = pd.DataFrame(data=interpolated_hardening_data.T, columns=pd.MultiIndex.from_tuples(column_headers))
    display(df)
    return df

def get_latex_yield_func_params(yield_func_name, yield_func_param_vals, val_format='.4f', pad_to=18):
    """Format yield function parameter symbols and values, given some macros defined
    in the manuscript.
    """
    
    vals = []
    latex_keys = []
    for k, v in yield_func_param_vals.items():            
        
        if k in ['yield_point', 'equivalent_stress', 'exponent']:
            continue
            
        vals.append(f'{v:{val_format}}')
        
        if yield_func_name == 'Barlat_Yld2004_18p':
            if '_p_' in k:
                new_k = f'\\yldFuncLinTransComp{{\\myprime}}{{{k[-2:]}}}'
            elif '_dp_' in k:
                new_k = f'\\yldFuncLinTransComp{{\\mydprime}}{{{k[-2:]}}}'
            else:
                raise ValueError(k)
            latex_keys.append(new_k)
                    
        elif yield_func_name == 'Barlat_Yld91':
            new_k = f'${k.upper()}$'
            latex_keys.append(new_k)
            
        elif yield_func_name == 'Hill1948':
            new_k = '$' + k + '_\\mathrm{h}$'
            latex_keys.append(new_k)            

        else:
            raise ValueError(f'Unknown yield_func_name: {yield_func_name}')
    
    if len(latex_keys) < pad_to:
        latex_keys += [''] * (pad_to - len(latex_keys))
        vals += ['-'] * (pad_to - len(vals))
    
    return np.array([latex_keys, vals])

def show_FLC(FLC):
    plt_data = []
    for strain_path_i in FLC['strain_paths']:
        name = f'BCs: {strain_path_i["displacement_BCs"]}; Groove angle: {strain_path_i["groove_angle_deg"]}'
        if strain_path_i['strain'] is None:
            continue
        plt_data.append({
            'x': strain_path_i['strain'][0],
            'y': strain_path_i['strain'][1],
            'mode': 'markers',
            'name': name,
        })

    plt_data.append({
        'x': FLC['forming_limits'][0],
        'y': FLC['forming_limits'][1],
        'mode': 'lines',
        'name': 'Forming limit',
    })

    fig_wig = graph_objects.FigureWidget(
        data=plt_data,
        layout={
            'width': 800,
            'xaxis': {
                'range': [-0.5, 0.5],
                'title': 'Minor strain',
            },
            'yaxis': {
                'range': [0, 0.5],
                'title': 'Major strain',
                'scaleanchor': 'x',
            }
        },
    )
    return fig_wig

def get_workflow_from_zenodo(root_folder, url, md5, name):
    """Download the workflow HDF5 file if it is not already present in root_folder."""
    root_folder = pathlib.Path(root_folder)

    data_folder = root_folder / "workflows"
    if not data_folder.is_dir():
        data_folder.mkdir()
    workflow_file = data_folder / name
    if not workflow_file.exists():
        with tqdm.tqdm(desc=f"Downloading workflow \"{name}\"", unit="bytes", unit_scale=True) as t:
            reporthook = dic_analysis.io.tqdm_hook(t)
            urllib.request.urlretrieve(url, workflow_file, reporthook=reporthook)
        if md5:
            if not dic_analysis.io.validate_checksum(workflow_file, md5):
                raise AssertionError("MD5 does not match: workflow file is corrupt. Delete workflow file and retry download.")
            else:
                print("MD5 validated. Download complete.")
                
    return workflow_file

## Download workflow HDF5 files from Zenodo

In [ ]:
all_workflow_paths = []
for wk_name, wk_info in dic_analysis.io.read_data_yaml('../data.yaml')['modelling_workflows'].items():
    wk_path_i = get_workflow_from_zenodo(root_folder='', name=wk_name + '.hdf5', **wk_info)    
    all_workflow_paths.append(wk_path_i)
(
    wkflow_1,
    wkflow_2,
    wkflow_3,
    wkflow_4,
    wkflow_5,
) = (
    load_workflow(i, full_path=True) for i in all_workflow_paths
)
    

## Workflow 1: Generate volume element

In [ ]:
print(wkflow_1)

## Workflow 2: Fit single-crystal parameters

In [ ]:
print(wkflow_2)

#### Show the experimental stress-strain data

In [ ]:
exp_tensile_test_dict = wkflow_2.tasks.get_tensile_test.elements[0].outputs.tensile_test
exp_tensile_test = TensileTest(**exp_tensile_test_dict)
exp_tensile_test.show(stress_strain_type='true')

#### Show the convergence of the stress-strain curve with iterations

Different elements in the `optimise_single_crystal_parameters` task correspond to different optimisation iterations. We would like to retrieve the element corresponding to the final iteration:

In [ ]:
final_iteration_element = wkflow_2.tasks.optimise_single_crystal_parameters.get_elements_from_iteration(-1)[0]

We can then reconstitue an `LMFitter` object from this element data, enabling a visualisation of the fitting process:

In [ ]:
lm_fitter_dict = final_iteration_element.outputs.levenberg_marquardt_fitter
lm_fitter = LMFitter.from_dict(lm_fitter_dict)
lm_fitter_fig = lm_fitter.show()
lm_fitter_fig

#### Produce a static plot for the manuscript

In [ ]:
lm_fitter_fig_static = plot_static_figure_single_crystal_fitting(lm_fitter)
lm_fitter_fig_static.write_image(str(FIG_EXPORT_DIR.joinpath('singleCrystalFitting.svg')))
lm_fitter_fig_static.show(config={'displayModeBar': False})

Use inkscape to generate a "compilable" figure for inclusion in the manuscript with: `inkscape -D --export-latex --export-type="pdf" singleCrystalFitting.svg`

#### Initial trial parameters

In [ ]:
initial_parameters = wkflow_2.tasks.simulate_volume_element_loading.elements[0].inputs.single_crystal_parameters
pretty_print_single_crystal_parameters(initial_parameters)

#### Final optimised parameters

Take the final parameters for which a set of simulations were run. This is the second-to-last iteration, because the last iteration generates new parameters that would be used for simulations in the next iteration.

In [ ]:
final_parameters = wkflow_2.tasks.optimise_single_crystal_parameters.get_elements_from_iteration(-2)[0].outputs.single_crystal_parameters
pretty_print_single_crystal_parameters(final_parameters)

## Workflow 3: Fit yield functions

In [ ]:
print(wkflow_3)

In [ ]:
all_load_responses = [
    LoadResponseSet.from_dict(i.outputs.fitted_yield_functions)
    for i in wkflow_3.tasks.fit_yield_function.elements
]

#### Tables of fitted yield function parameters at all yield points

In [ ]:
all_fitted_params = show_all_fitted_yield_function_parameters(all_load_responses)

#### Example of accessing the parameters from a given yield point fit

In [ ]:
all_fitted_params['Barlat_Yld2004_18p'][10]

#### Yield function evolution - animation

In [ ]:
animate_yield_function_evolution(all_load_responses[1:], plane=[0,0,1], normalise=True, sheet_dirs=SHEET_DIRS)

#### Produce a static plot for the manuscript

In [ ]:
yield_funcs_fig_static = plot_static_figure_yield_function_type_comparison(all_load_responses[1:], yield_point=0.00275)
yield_funcs_fig_static.write_image(str(FIG_EXPORT_DIR.joinpath('yieldFuncComparison.svg')))
yield_funcs_fig_static.show(config={'displayModeBar': False})

Use inkscape to generate a "compilable" figure for inclusion in the manuscript with: `inkscape -D --export-latex --export-type="pdf" yieldFuncComparison.svg`

#### Generate a parameter table for manuscript

In [ ]:
# Index 10 is the 0.00275 yield point
data = np.concatenate([
    get_latex_yield_func_params('Barlat_Yld2004_18p', all_fitted_params['Barlat_Yld2004_18p'][10], pad_to=18),
    get_latex_yield_func_params('Barlat_Yld91', all_fitted_params['Barlat_Yld91'][10], pad_to=18),
    get_latex_yield_func_params('Hill1948', all_fitted_params['Hill1948'][10], pad_to=18),        
]).T
print(pd.DataFrame(data=data).to_latex(header=False, index=False, escape=False))

#### Yield function evolution at selected yield points

##### Von Mises

In [ ]:
compare_yield_function_yield_points(all_load_responses[0], slice(0, -1, 10), plane=[0, 0, 1], sheet_dirs=SHEET_DIRS)

##### Hill 1948

In [ ]:
compare_yield_function_yield_points(all_load_responses[1], slice(0, -1, 10), plane=[0, 0, 1], sheet_dirs=SHEET_DIRS)

##### Barlat Yld91

In [ ]:
compare_yield_function_yield_points(all_load_responses[2], slice(0, -1, 10), plane=[0, 0, 1], sheet_dirs=SHEET_DIRS)

##### Barlat Yld2004-18p

In [ ]:
compare_yield_function_yield_points(all_load_responses[3], slice(0, -1, 10), plane=[0, 0, 1], sheet_dirs=SHEET_DIRS)

## Workflow 4: Estimate hardening curves

In [ ]:
print(wkflow_4)

#### Plot plastic stress-strain curves and hardening rates for each strain path

#### Produce a static plot for the manuscript

#### Interpolate values of stress for each strain path

In [ ]:
hardening_curve_tasks = {task.context: task for task in wkflow_4.tasks if task.name == 'simulate_volume_element_loading'}
hardening_data = collect_hardening_data(hardening_curve_tasks, yield_stress=95e6)
interpolated_data = show_interpolated_plastic_stress_strain_data(hardening_data)

#### Write out plastic stress-strain data for Abaqus workflow (Workflow 5)

In [ ]:
for strain_path in hardening_data.keys():
    path_i = PLASTIC_TABLES_DIR.joinpath(f'{strain_path}.csv')
    with open(path_i, 'w') as file:
        file.write('% Plastic table ({} strain path)\n% True stress (MPa), True strain\n'.format(strain_path))
        ordered_cols_df = interpolated_data.get((strain_path)).reindex(columns=interpolated_data.get((strain_path)).columns[::-1])    
        ordered_cols_df['interpolated_stress'] /= 1e6
        ordered_cols_df.to_csv(file, header=False, index=False, line_terminator='\n', float_format='%.5f')

## Workflow 5: Simulate Marciniak-Kuzynski analysis

In [ ]:
FLC = wkflow_5.tasks.find_forming_limit_curve.elements[0].outputs.forming_limit_curve

In [ ]:
show_FLC(FLC)